## Load and Process Data:

Read the questions and answers from your faqs.csv file.
Create a list of questions and answers.

In [1]:
import pandas as pd

# Load the FAQ data
df = pd.read_csv('faqs.csv')
df = df.drop_duplicates(subset=['Question', 'Answer', 'Category'], keep='first')


questions = df['Question'].tolist()
answers = df['Answer'].tolist()
categories = df["Category"].tolist()

In [2]:
df.shape

(1058, 3)

In [3]:
df.head()

,Category,Question,Answer
0,FAQ BMBF-Förderung,Umfassen die Ausgaben für Literatur in der Pos...,Wenn die kostenpflichtige Nutzung von Literatu...
1,FAQ BMBF-Förderung,Ab welchem Betrag müssen Geräte abgeschrieben ...,Bitte benutzen Sie zur Beantwortung dieser Fra...
2,FAQ BMBF-Förderung,Ab welcher Grenze müssen bei einer Beauftragun...,Der Projektträger kann an dieser Stelle leider...
3,FAQ BMBF-Förderung,Ab wieviel Mitarbeitern gilt eine Firma als Gr...,Für die Einstufung als KMU ( meist ab 250 Pers...
4,FAQ BMBF-Förderung,An wen kann ich mich für eine Antragsberatung ...,Ansprechpersonen für eine Antragsberatung sind...


### Generate Embeddings

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained sentence transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for questions
def generate_embeddings(questions):
    embeddings = model.encode(questions)
    return embeddings


/usr/local/anaconda3/envs/vdi_chatbot_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/anaconda3/envs/vdi_chatbot_py310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Set Up Elasticsearch:

- Install and run Elasticsearch locally or use a cloud service.
- Create an index for storing your FAQ data.

#### Index Data:

Use the Elasticsearch Python client to index your data.

In [6]:
from elasticsearch import Elasticsearch, helpers

# Initialize Elasticsearch client
es = Elasticsearch("http://localhost:9200")


In [7]:
# delete index if available
index_name = "faq_index"
es.indices.delete(index=index_name, ignore_unavailable=True)


ObjectApiResponse({'acknowledged': True})

In [8]:


# Define the index mapping to support dense vectors
index_mapping = {
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "category": {"type": "keyword"},
            "question_embedding": {
                "type": "dense_vector",
                "dims": 384  # Adjust this according to your model output size
            }
        }
    }
}

# Create an index
es.indices.create(index='faq_index', body=index_mapping, ignore=400)


/var/folders/81/16mr_0gj603btxm2xxhl704c0000gn/T/ipykernel_75581/4151266374.py:17: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='faq_index', body=index_mapping, ignore=400)
/var/folders/81/16mr_0gj603btxm2xxhl704c0000gn/T/ipykernel_75581/4151266374.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='faq_index', body=index_mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_index'})

In [9]:

# Prepare data for indexing
def generate_documents(questions, answers, categories, embeddings):
    for question, answer, category, embedding in zip(questions, answers, categories, embeddings):
        yield {
            "_index": "faq_index",
            "_source": {
                "category": category,
                "question": question,
                "answer": answer,
                "question_embedding": embedding.tolist()  # Convert to list for JSON storage
            }
        }


In [10]:
embeddings = generate_embeddings(questions)

In [11]:

# Index data
helpers.bulk(es, generate_documents(questions, answers, categories, embeddings))

(1058, [])

In [12]:
df[df['Question'] == "Bis wann kann die Mittelverschiebung beantragt werden?"] # there are duplicate questions which find themselves in multiple categories...


,Category,Question,Answer
7,FAQ BMBF-Förderung,Bis wann kann die Mittelverschiebung beantragt...,Eine Mittelverschiebung von nicht benötigten M...
655,-Projektablauf,Bis wann kann die Mittelverschiebung beantragt...,Eine Mittelverschiebung von nicht benötigten M...
729,--Projektablauf auf Kostenbasis (AZK),Bis wann kann die Mittelverschiebung beantragt...,Eine Mittelverschiebung von nicht benötigten M...
782,---Zahlungsanforderung,Bis wann kann die Mittelverschiebung beantragt...,Eine Mittelverschiebung von nicht benötigten M...
843,--Projektablauf auf Ausgabenbasis (AZA/AZAP),Bis wann kann die Mittelverschiebung beantragt...,Eine Mittelverschiebung von nicht benötigten M...


### Set Up the Retrieval Mechanism
#### Create a Search Function:

Use Elasticsearch to retrieve relevant questions based on user input.


In [13]:
def search_question(user_query):
    response = es.search(
        index="faq_index",
        body={
            "query": {
                "multi_match": {
                    "query": user_query,
                    "fields": ["question^4", "category"],  # Boost for 'question'
                    "type": "most_fields"  # Most relevant field wins
                }
            }
        }
    )
    return response['hits']['hits']


In [15]:
# user_query = "Wie lange kann ich eine Mittelverschiebung beantragen?"
user_query = "Bis wann kann die Mittelverschiebung beantragt werden?"

search_question(user_query)

/var/folders/81/16mr_0gj603btxm2xxhl704c0000gn/T/ipykernel_75581/3929286585.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(


[{'_index': 'faq_index',
  '_id': 'IUJxBpIBF5Gllf7ErR86',
  '_score': 80.389465,
  '_source': {'category': 'FAQ BMBF-Förderung',
   'question': 'Bis wann kann die Mittelverschiebung beantragt werden?',
   'answer': 'Eine Mittelverschiebung von nicht benötigten Mitteln im aktuellen Haushaltsjahr ist spätestens mit der letzten Zahlungsanforderung eines Kalenderjahres zu beantragen. Der Termin zur Einreichung der …',
   'question_embedding': [-0.06885983794927597,
    0.060505062341690063,
    0.002912599593400955,
    -0.00399744650349021,
    -0.05336926877498627,
    0.04853307083249092,
    0.01002613827586174,
    0.07113131880760193,
    0.019125226885080338,
    -0.03351515159010887,
    0.08569822460412979,
    -0.058446191251277924,
    0.024713613092899323,
    -0.06942953169345856,
    0.010201658122241497,
    -0.08501966297626495,
    -0.02912805788218975,
    0.151739239692688,
    -0.02892560325562954,
    -0.028474140912294388,
    0.015276849269866943,
    -0.019348291680

In [18]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


from openai import OpenAI
client = OpenAI()

# Load your OpenAI API key
client.api_key = os.getenv("OPENAI_API_KEY")

### Create the RAG-based Chatbot
#### Combine Retrieval and Generation:

Retrieve relevant answers from Elasticsearch and generate a response using the language model.


In [22]:
import os


# Function to generate a response using OpenAI's GPT-3.5 or GPT-4
def generate_answer(user_query, best_match):
    prompt = f"Q: {user_query}\nA: {best_match}"
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Use "gpt-4" if you have access to GPT-4
        messages=[
            {"role": "system", "content": "You are an assistant that answers questions based on the given information."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response



In [24]:
# Example usage
user_query = "Darf das Konsortium nur aus Forschungseinrichtungen bestehen?"
best_match = "Bitte entnehmen Sie die Information zu den Antragsberechtigten der jeweiligen Förderbekanntmachung."
answer = generate_answer(user_query, best_match)


In [31]:
answer.choices[0].message.content


'Die Zusammensetzung des Konsortiums, ob es nur aus Forschungseinrichtungen bestehen darf oder auch weitere Akteure beinhalten kann, hängt von den Anforderungen der jeweiligen Förderbekanntmachung ab. Bitte überprüfen Sie die spezifischen Richtlinien und Bedingungen der Förderung, um festzustellen, wer berechtigt ist, Anträge zu stellen.'